In [ ]:
#r "nuget:Kokkos.NET"

In [ ]:
using Kokkos;
using System;
using System.Diagnostics;
using System.IO;
using System.Reflection;
using System.Runtime.CompilerServices;
using System.Security;
using System.Text;
using System.Transactions;

In [ ]:
ExecutionSpaceKind ExecutionSpace = ExecutionSpaceKind.OpenMP;

try
{
    Devices devices = new Devices();
    
    int gpuCount = devices.Gpu.Count;
    
    if(gpuCount > 0)
    {
        display($"Gpu Count: {gpuCount}");
        
        int version = devices.Gpu.First().Architecture.Version;
        
        display($"Gpu Arch: {devices.Gpu.First().Architecture.Name} {version}");
        
        if(version >= 520)
        {
            ExecutionSpace = ExecutionSpaceKind.Cuda;
        }
    }
}
catch(Exception e)
{
    display(e.Message);
}

In [ ]:
int  num_threads      = 4;
int  num_numa         = 1;
int  device_id        = 0;
int  ndevices         = 1;
int  skip_device      = 9999;
bool disable_warnings = false;

InitArguments arguments = new InitArguments(num_threads,
                                            num_numa,
                                            device_id,
                                            ndevices,
                                            skip_device,
                                            disable_warnings);



using(ScopeGuard sg = new ScopeGuard(arguments))
{
    if(ExecutionSpace == ExecutionSpaceKind.Cuda)
    {
        View<double, Cuda> view = new View<double, Cuda>("view", 10);

        view[0] = 1321.258;
        view[1] = 123123.12;

        double value0 = view[0];
        double value1 = view[1];

        if(!(Math.Abs(value0 - 1321.258) <= double.Epsilon))
        {
            throw new Exception();
        }

        if(!(Math.Abs(value1 - 123123.12) <= double.Epsilon))
        {
            throw new Exception();
        }
    }
    else
    {
        View<double, OpenMP> view = new View<double, OpenMP>("view", 10);

        view[0] = 1321.258;
        view[1] = 123123.12;

        double value0 = view[0];
        double value1 = view[1];

        if(!(Math.Abs(value0 - 1321.258) <= double.Epsilon))
        {
            throw new Exception();
        }

        if(!(Math.Abs(value1 - 123123.12) <= double.Epsilon))
        {
            throw new Exception();
        }
    }
}